In [9]:
import opstrat as op
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
import seaborn as sns

import json
with open('config.json') as config_file:
     data = json.load(config_file)
print(data)

last_price          = data['Current_Price']
annual_drift        = data['Annual_Drift']
annual_volatility   = data['Annual_Volatiltiy']
time_period         = data['Time_Period']
simu                = data['No_of_days']
r                   = data['rate_return']
sigma               = data['sigma']
price               = data['Current_Price']

df = pd.DataFrame()

last_price_list = [last_price]
diff_list       = []
tenor_list      = [0.083333333333333333]
count           = 0
count_list      = []
while count < simu:
    count = count + 1
    difference = last_price*annual_drift*time_period + last_price*annual_volatility*(time_period**(0.5))*norm.ppf(random.random())

    last_price = last_price + difference
    tenor      = ((23-count)/(23*12))
    last_price_list.append(last_price)
    diff_list.append(difference)
    count_list.append(count)
    tenor_list.append(tenor)
last_price_list.pop(-1)
tenor_list.pop(-1)
df['Count'] = count_list
df['Price'] = last_price_list
df['Change_price'] = diff_list
df['tenor'] = tenor_list

{'Current_Price': 17689, 'Annual_Drift': 0.07, 'Annual_Volatiltiy': 0.15, 'Time_Period': 0.003968254, 'rate_return': 0.069, 'sigma': 0.1279, 'No_of_days': 21}


In [10]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import math


#calculation of probabiltiy of hitting nearest strike price 
Trading_Days = 252
Expiry_Days = 21
Daily_volatility = sigma*math.sqrt(Expiry_Days/Trading_Days)
df['Sigma']      = Daily_volatility*df['Price']




list1 = [int(round(price, -2))+900,int(round(price, -2))+600]
list2 = [int(round(price, -2))-800,int(round(price, -2))-600]
datalist = dict()
for k,p in zip(list1,list2):
    call_option     =   []
    call_thetalist  =   []
    call_gammalist  =   []
    call_deltalist  =   []
    call_rholist    =   []
    call_vegalist   =   [] 
    Prob_call_list       =   []
    Prob_put_list        =   []

    put_option      =   []
    put_thetalist   =   []
    put_gammalist   =   []
    put_deltalist   =   []
    put_rholist     =   []
    put_vegalist    =   []

    for last_price,T,sig in zip(df['Price'],df['tenor'],df['Sigma']):

        d1         = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2         = d1-sigma*(T**0.5)
        N_d1       = norm.cdf(d1)
        N_d2       = norm.cdf(d2)
        N_d1_      = 1-norm.cdf(d1)
        N_d2_      = 1-norm.cdf(d2)
        call       = last_price*N_d1-k*math.exp(-r*T)*N_d2
        #put        = k*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
        call_delta = norm.cdf(d1)
        call_gamma = norm.pdf(d1)/(last_price*sigma*sqrt(T))
        call_vega  = (last_price*norm.pdf(d1)*sqrt(T))
        call_theta = ((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
        call_rho   = (k*T*exp(-r*T)*norm.cdf(d2))
        
        
        Prob_call = norm.pdf((k-last_price)/sig)


        Prob_call_list.append(Prob_call)
        call_option.append(call)
        call_thetalist.append(call_theta)
        call_gammalist.append(call_gamma)
        call_deltalist.append(call_delta)
        call_rholist.append(call_rho)
        call_vegalist.append(call_vega)

        datalist['Prob_call_' + str(k)] =   Prob_call_list
        datalist['CE_'+str(k)]          =   call_option
        datalist['theta_CE'+str(k)]     =   call_thetalist
        datalist['gamma_CE'+str(k)]     =   call_gammalist
        datalist['delta_CE'+str(k)]     =   call_deltalist
        datalist['rho_CE'+str(k)]       =   call_rholist
        datalist['vega_CE'+str(k)]      =   call_vegalist


        d1         = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2         = d1-sigma*(T**0.5)
        N_d1       = norm.cdf(d1)
        N_d2       = norm.cdf(d2)
        N_d1_      = 1-norm.cdf(d1)
        N_d2_      = 1-norm.cdf(d2)
        call       = last_price*N_d1-p*math.exp(-r*T)*N_d2
        put        = p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
        put_delta  = -norm.cdf(-d1)
        put_gamma  = norm.pdf(d1)/(last_price*sigma*sqrt(T))
        put_vega   = (last_price*norm.pdf(d1)*sqrt(T))
        put_theta  = (-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
        put_rho    = (-p*T*exp(-r*T)*norm.cdf(-d2))
        Prob_put = norm.pdf((p-last_price)/sig)

        Prob_put_list.append(Prob_put)
        put_option.append(put)
        put_thetalist.append(put_theta)
        put_gammalist.append(put_gamma)
        put_deltalist.append(put_delta)
        put_rholist.append(put_rho)
        put_vegalist.append(put_vega)

        datalist['Prob_put_' + str(p)] =   Prob_put_list
        datalist['PE_'+str(p)]         = put_option
        datalist['theta_PE_'+str(p)]   = put_thetalist
        datalist['gamma_PE_'+str(p)]   = put_gammalist
        datalist['delta_PE_'+str(p)]   = put_deltalist
        datalist['rho_PE_'+str(p)]     = put_rholist
        datalist['vega_PE_'+str(p)]    = put_vegalist
        
      
df1 = pd.DataFrame(datalist)

df = pd.concat([df,df1],axis=1)


In [11]:
#Changing the signs of greeks according to options selling and buying
df['PE_'+str((round(price, -2))-800)] = - df['PE_'+str((round(price, -2))-800)]
df['CE_'+str((round(price, -2))+900)] = - df['CE_'+str((round(price, -2))+900)]


put_sell = ['theta_PE_','gamma_PE_','delta_PE_','rho_PE_','vega_PE_']
for data in put_sell:
    df[data+str((round(price, -2))-600)] = - df[data+str((round(price, -2))-600)]

call_sell = ['theta_CE','gamma_CE','delta_CE','rho_CE','vega_CE']
for data in call_sell:
    df[data+str((round(price, -2))+600)] = - df[data+str((round(price, -2))+600)]




#Calculating the total premium paid or recieved 
filter_col = [col for col in list(df) if col.startswith(('PE_','CE_'))]
df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
for d in df[filter_col]:
    df['Total_Prem'] = df[filter_col].sum(axis=1)   # Total_PP = total put premium 


#Calculating the total Profit and Loss

df['Total_P&L'] = df['Total_Prem'][0] - df['Total_Prem']


#Calculating greeks for overall data

list1 = ['theta','gamma','delta','rho','vega']

datalist4 = dict()

for data in list1:
    filter_col = [col for col in list(df) if col.startswith(data)]
    df['Port_'+ data] = 0 # initialize value, otherwise throws KeyErrorx
    for d in df[filter_col]:
        df['Port_'+ data] = df[filter_col].sum(axis=1)    

#final_df = pd.concat([df,df],axis = 1)

In [12]:
df.to_csv('manual.csv')

In [13]:
df1 = df[df['Prob_call_18300'] <= 0.30]

In [14]:
df

,Count,Price,Change_price,tenor,Sigma,Prob_call_18600,CE_18600,theta_CE18600,gamma_CE18600,delta_CE18600,...,delta_PE_17100,rho_PE_17100,vega_PE_17100,Total_Prem,Total_P&L,Port_theta,Port_gamma,Port_delta,Port_rho,Port_vega
0,1,17689.000000,-176.298200,0.083333,653.105293,0.150804,-37.122898,-915.052867,0.000302,0.117822,...,0.137549,206.619633,-1122.968988,70.229186,0.000000,751.899569,-0.000271,-0.051866,-70.602174,-903.623885
1,2,17512.701800,97.190563,0.079710,646.596090,0.097025,-18.369995,-595.329669,0.000206,0.067101,...,0.203045,289.276526,-1396.822373,62.834671,7.394515,705.674538,-0.000283,-0.001023,3.581054,-883.448200
2,3,17609.892362,96.467549,0.076087,650.184516,0.125129,-23.262624,-724.479416,0.000246,0.083174,...,0.158827,216.817652,-1176.205803,61.482073,8.747112,786.017769,-0.000298,-0.027273,-31.864285,-900.814139
3,4,17706.359912,13.032160,0.072464,653.746248,0.156732,-29.297290,-874.556891,0.000293,0.102459,...,0.120094,156.738228,-953.995576,62.492753,7.736433,845.278325,-0.000305,-0.055375,-66.521450,-886.037099
4,5,17719.392072,3.850716,0.068841,654.227415,0.161245,-27.446360,-871.866978,0.000293,0.099047,...,0.111494,138.243445,-882.690590,60.110637,10.118549,880.869133,-0.000317,-0.058906,-67.716328,-875.169900
5,6,17723.242788,-69.146435,0.065217,654.369590,0.162588,-24.698219,-848.497193,0.000287,0.092722,...,0.105630,124.029987,-826.578681,57.082510,13.146676,916.903612,-0.000330,-0.059618,-65.187436,-864.740087
6,7,17654.096353,228.433117,0.061594,651.816596,0.139192,-16.425291,-675.137556,0.000233,0.067519,...,0.124170,137.175523,-897.741843,50.446080,19.783106,930.068450,-0.000350,-0.034923,-34.867323,-858.185763
7,8,17882.529470,267.258414,0.057971,660.250701,0.221052,-34.981133,-1154.798281,0.000382,0.128793,...,0.054985,57.755325,-478.862961,63.874793,6.354392,977.073545,-0.000322,-0.112946,-113.384759,-763.383835
8,9,18149.787884,125.027107,0.054348,670.118296,0.318344,-78.653281,-1880.095162,0.000585,0.247905,...,0.016222,16.174150,-171.378726,101.500008,-31.270822,742.239914,-0.000191,-0.187529,-179.462696,-437.157150
9,10,18274.814990,247.565679,0.050725,674.734490,0.355200,-106.371138,-2240.303959,0.000677,0.316898,...,0.007291,6.823485,-83.144948,124.286175,-54.056989,543.432388,-0.000104,-0.213090,-191.226363,-224.719186


In [15]:
# datalist2 = dict()
# list2 = [int(round(price, -2))-800,int(round(price, -2))-600]
# for k in list2:
    
#     put_option     = []
#     put_thetalist  = []
#     put_gammalist  = []
#     put_deltalist  = []
#     put_rholist    = []
#     put_vegalist   = [] 

#     for last_price,T in zip(df['Price'],df['tenor']):
#         d1         = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#         d2         = d1-sigma*(T**0.5)
#         N_d1       = norm.cdf(d1)
#         N_d2       = norm.cdf(d2)
#         N_d1_      = 1-norm.cdf(d1)
#         N_d2_      = 1-norm.cdf(d2)
#         call       = last_price*N_d1-k*math.exp(-r*T)*N_d2
#         put        = k*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
#         put_delta = -norm.cdf(-d1)
#         put_gamma = norm.pdf(d1)/(last_price*sigma*sqrt(T))
#         put_vega  = (last_price*norm.pdf(d1)*sqrt(T))
#         put_theta = (-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*k*exp(-r*T)*norm.cdf(-d2))
#         put_rho   = (-k*T*exp(-r*T)*norm.cdf(-d2))
        
#         put_option.append(put)
#         put_thetalist.append(put_theta)
#         put_gammalist.append(put_gamma)
#         put_deltalist.append(put_delta)
#         put_rholist.append(put_rho)
#         put_vegalist.append(put_vega)

#         datalist2['PE_'+str(k)]      = put_option
#         datalist2['theta_'+str(k)]   = put_thetalist
#         datalist2['gamma_'+str(k)]   = put_gammalist
#         datalist2['delta_'+str(k)]   = put_deltalist
#         datalist2['rho_'+str(k)]     = put_rholist
#         datalist2['vega_'+str(k)]    = put_vegalist
        
# df2 = pd.DataFrame(datalist2)




In [16]:
# Trading_Days = 252
# Expiry_Days = 23


# Daily_volatility = sigma*math.sqrt(Expiry_Days/Trading_Days)
# df['Sigma'] = Daily_volatility*df['Price']

# K = [int(round(price, -2))-800,int(round(price, -2))-600,int(round(price, -2))+600,int(round(price, -2))+900]
# datalists = dict()
# for k in K:
#     prob_list = []
#     for price,sig in zip(df['Price'],df['Sigma']):
#         prob = norm.pdf((k-price)/sig)
#         prob_list.append(prob)
       
#     datalists['Prob_'+str(k)] = prob_list

    
# df2 = pd.DataFrame(datalists)

# df = pd.concat([df,df2],axis = 1)